# Analyse Complète: Market Data → Heston Calibration → Monte Carlo → IV Surfaces

Ce notebook effectue une analyse complète en 5 étapes:
1. **Téléchargement des données** de yfinance
2. **Heatmap des prix market** (grille S × K)
3. **IV Surface des données market** en 3D
4. **Calibration Heston** via réseau de neurones PyTorch
5. **Heatmap des prix Heston** (Monte Carlo)
6. **IV Surface BS** inversée à partir des prix Heston en 3D

## 1. Imports et Configuration

In [26]:
from IPython.display import display
from __future__ import annotations

import math
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import yfinance as yf

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
DEVICE = torch.device("cpu")
MIN_IV_MATURITY = 0.1

print("✓ Imports réussis")

✓ Imports réussis


## 2. Configuration des Paramètres

In [27]:
# Paramètres de marché
TICKER = "SPY"
RF_RATE = 0.02
MATURITY = 1.0  # 1 an

# Paramètres de la grille (S0 ± 10, step de 1)
SPAN = 10.0
STEP = 1.0

# Paramètres calibration (augmenté pour utiliser plus de données)
MAX_POINTS = 1000  # Augmenté pour profiter de toutes les données
MAX_ITERS = 150    # Plus d'itérations pour meilleure convergence
LR = 5e-3

# Paramètres Monte Carlo
N_PATHS = 50000
N_STEPS = 100

print(f"Configuration:")
print(f"  Ticker: {TICKER}")
print(f"  Maturity: {MATURITY} ans")
print(f"  Grille: S0 ± {SPAN}, step = {STEP}")
print(f"  Monte Carlo: {N_PATHS:,} trajectoires, {N_STEPS} pas")

Configuration:
  Ticker: SPY
  Maturity: 1.0 ans
  Grille: S0 ± 10.0, step = 1.0
  Monte Carlo: 50,000 trajectoires, 100 pas


## 3. Fonctions Utilitaires

In [28]:
def fetch_spot(symbol: str) -> float:
    """Récupère le prix spot actuel."""
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d")
    if hist.empty:
        raise RuntimeError("Unable to retrieve spot price.")
    return float(hist["Close"].iloc[-1])


def download_all_options(symbol: str, years_ahead: float = 2.5) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Télécharge TOUTES les options disponibles pour maximiser les données d'entraînement."""
    ticker = yf.Ticker(symbol)
    spot = fetch_spot(symbol)
    expirations = ticker.options
    
    if not expirations:
        raise RuntimeError(f"No option expirations found for {symbol}")
    
    now = pd.Timestamp.utcnow().tz_localize(None)
    limit_date = now + pd.Timedelta(days=365 * years_ahead)
    
    calls_data = []
    puts_data = []
    
    print(f"Téléchargement de toutes les expirations jusqu'à {years_ahead} ans...")
    for idx, exp in enumerate(expirations):
        exp_dt = pd.Timestamp(exp)
        if exp_dt > limit_date:
            continue
            
        T = max((exp_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)
        if T < 0.01:  # Ignorer les expirations très proches
            continue
        
        try:
            chain = ticker.option_chain(exp)
            
            # Calls
            for _, row in chain.calls.iterrows():
                K = float(row["strike"])
                # Filtre: seulement les strikes dans S0 ± 50 pour réduire le bruit
                if abs(K - spot) <= 50:
                    calls_data.append({
                        "S0": spot,
                        "K": K,
                        "T": T,
                        "C_mkt": float(row["lastPrice"]),
                        "iv_market": float(row.get("impliedVolatility", float("nan"))),
                        "option_type": "call",
                        "expiry": exp
                    })
            
            # Puts
            for _, row in chain.puts.iterrows():
                K = float(row["strike"])
                if abs(K - spot) <= 50:
                    puts_data.append({
                        "S0": spot,
                        "K": K,
                        "T": T,
                        "P_mkt": float(row["lastPrice"]),
                        "iv_market": float(row.get("impliedVolatility", float("nan"))),
                        "option_type": "put",
                        "expiry": exp
                    })
            
            if (idx + 1) % 5 == 0:
                print(f"  Processed {idx + 1}/{len(expirations)} expirations...")
                
        except Exception as e:
            print(f"  Warning: Failed to download {exp}: {e}")
            continue
    
    calls_df = pd.DataFrame(calls_data)
    puts_df = pd.DataFrame(puts_data)
    
    # Nettoyer les données
    calls_df = calls_df[calls_df["C_mkt"] > 0]
    puts_df = puts_df[puts_df["P_mkt"] > 0]
    calls_df = calls_df[~calls_df["iv_market"].isna()]
    puts_df = puts_df[~puts_df["iv_market"].isna()]
    
    return calls_df, puts_df


def plot_heatmap_2d(data: np.ndarray, K_grid: np.ndarray, S_grid: np.ndarray, title: str, zlabel: str = "Value"):
    """Crée une heatmap 2D."""
    fig = go.Figure(
        data=go.Heatmap(
            z=data,
            x=K_grid,
            y=S_grid,
            colorscale="Viridis",
            colorbar=dict(title=zlabel),
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title="Strike K",
        yaxis_title="Spot S",
        width=700,
        height=600,
    )
    return fig


def plot_surface_3d(S_grid: np.ndarray, K_grid: np.ndarray, data: np.ndarray, title: str, zlabel: str = "Value"):
    """Crée une surface 3D."""
    K_mesh, S_mesh = np.meshgrid(K_grid, S_grid)
    
    fig = go.Figure(
        data=go.Surface(
            x=K_mesh,
            y=S_mesh,
            z=data,
            colorscale="Viridis",
            colorbar=dict(title=zlabel),
        )
    )
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="Strike K",
            yaxis_title="Spot S",
            zaxis_title=zlabel,
        ),
        width=800,
        height=700,
    )
    return fig

print("✓ Fonctions utilitaires définies")

✓ Fonctions utilitaires définies


## 4. Fonctions Black-Scholes

In [29]:
def bs_price(S0: float, K: float, T: float, vol: float, r: float, option_type: str = "call") -> float:
    """Prix Black-Scholes."""
    if T <= 0.0 or vol <= 0.0 or S0 <= 0.0 or K <= 0.0:
        if option_type == "call":
            return max(0.0, S0 - K * math.exp(-r * T))
        else:
            return max(0.0, K * math.exp(-r * T) - S0)
    
    sqrt_T = math.sqrt(T)
    d1 = (math.log(S0 / K) + (r + 0.5 * vol * vol) * T) / (vol * sqrt_T)
    d2 = d1 - vol * sqrt_T
    
    nd1 = 0.5 * (1.0 + math.erf(d1 / math.sqrt(2.0)))
    nd2 = 0.5 * (1.0 + math.erf(d2 / math.sqrt(2.0)))
    
    if option_type == "call":
        return S0 * nd1 - K * math.exp(-r * T) * nd2
    else:
        return K * math.exp(-r * T) * (1 - nd2) - S0 * (1 - nd1)


def implied_vol_from_price(
    price: float, S0: float, K: float, T: float, r: float, option_type: str = "call",
    tol: float = 1e-6, max_iter: int = 100
) -> float:
    """Calcule la volatilité implicite par bissection."""
    if option_type == "call":
        intrinsic = max(0.0, S0 - K * math.exp(-r * T))
    else:
        intrinsic = max(0.0, K * math.exp(-r * T) - S0)
    
    if price <= intrinsic + 1e-12:
        return 0.0
    
    low, high = 1e-6, 3.0
    p_high = bs_price(S0, K, T, high, r, option_type)
    
    while p_high < price and high < 10.0:
        high *= 2.0
        p_high = bs_price(S0, K, T, high, r, option_type)
    
    if p_high < price:
        return float("nan")
    
    for _ in range(max_iter):
        mid = 0.5 * (low + high)
        p_mid = bs_price(S0, K, T, mid, r, option_type)
        
        if abs(p_mid - price) < tol:
            return mid
        
        if p_mid > price:
            high = mid
        else:
            low = mid
    
    return 0.5 * (low + high)

print("✓ Fonctions Black-Scholes définies")

✓ Fonctions Black-Scholes définies


## 5. Fonctions de Calibration Heston

In [30]:
def prices_from_unconstrained(
    u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, r: float, q: float
) -> torch.Tensor:
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        price_i = carr_madan_call_torch(S0_i, r, q, T_i, params, K_i)
        prices.append(price_i)
    return torch.stack(prices)


def loss(u: torch.Tensor, S0_t: torch.Tensor, K_t: torch.Tensor, T_t: torch.Tensor, C_mkt_t: torch.Tensor, r: float, q: float) -> torch.Tensor:
    model_prices = prices_from_unconstrained(u, S0_t, K_t, T_t, r, q)
    return torch.mean((model_prices - C_mkt_t) ** 2)


def calibrate_heston(calls_df: pd.DataFrame, r: float, q: float, max_points: int, max_iters: int, lr: float) -> dict:
    """Calibre les paramètres Heston sur les calls."""
    df = calls_df.copy()
    df = df[df["T"] >= MIN_IV_MATURITY]
    df = df[df["C_mkt"] > 0]
    df = df.sample(n=min(len(df), max_points), random_state=42)

    S0_t = torch.tensor(df["S0"].values, dtype=torch.float64, device=DEVICE)
    K_t = torch.tensor(df["K"].values, dtype=torch.float64, device=DEVICE)
    T_t = torch.tensor(df["T"].values, dtype=torch.float64, device=DEVICE)
    C_mkt_t = torch.tensor(df["C_mkt"].values, dtype=torch.float64, device=DEVICE)

    u = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0], dtype=torch.float64, device=DEVICE, requires_grad=True)
    optimizer = torch.optim.Adam([u], lr=lr)

    print(f"Calibration sur {len(df)} points...")
    print(f"Lancement de l'entraînement du réseau de neurones...\n")
    for it in range(max_iters):
        optimizer.zero_grad()
        L = loss(u, S0_t, K_t, T_t, C_mkt_t, r, q)
        L.backward()
        optimizer.step()
        
        if (it + 1) % 10 == 0:
            print(f"  🔄 NN Training | Epoch {it+1:3d}/{max_iters} | Loss = {L.item():.6e}")

    params_final = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    calib = {
        "kappa": float(params_final.kappa.cpu()),
        "theta": float(params_final.theta.cpu()),
        "sigma": float(params_final.sigma.cpu()),
        "rho": float(params_final.rho.cpu()),
        "v0": float(params_final.v0.cpu()),
    }
    
    return calib


def params_from_calib(calib: dict) -> HestonParams:
    """Convertit dict en HestonParams."""
    return HestonParams(
        kappa=torch.tensor(calib["kappa"], dtype=torch.float64, device=DEVICE),
        theta=torch.tensor(calib["theta"], dtype=torch.float64, device=DEVICE),
        sigma=torch.tensor(calib["sigma"], dtype=torch.float64, device=DEVICE),
        rho=torch.tensor(calib["rho"], dtype=torch.float64, device=DEVICE),
        v0=torch.tensor(calib["v0"], dtype=torch.float64, device=DEVICE),
    )

print("✓ Fonctions de calibration définies")

✓ Fonctions de calibration définies


## 6. Fonctions Monte Carlo Heston

In [31]:
def heston_monte_carlo_price(
    S0: float, K: float, T: float, r: float, q: float,
    params: HestonParams, n_paths: int, n_steps: int, option_type: str = "call"
) -> float:
    """Price option avec Monte Carlo Heston."""
    dt = T / n_steps
    sqrt_dt = np.sqrt(dt)
    
    kappa = float(params.kappa.cpu())
    theta = float(params.theta.cpu())
    sigma = float(params.sigma.cpu())
    rho = float(params.rho.cpu())
    v0 = float(params.v0.cpu())
    
    S = np.ones(n_paths) * S0
    v = np.ones(n_paths) * v0
    
    for _ in range(n_steps):
        Z1 = np.random.standard_normal(n_paths)
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.standard_normal(n_paths)
        
        v_sqrt = np.sqrt(np.maximum(v, 0))
        v = v + kappa * (theta - v) * dt + sigma * v_sqrt * sqrt_dt * Z2
        v = np.maximum(v, 0)
        
        S = S * np.exp((r - q - 0.5 * v) * dt + v_sqrt * sqrt_dt * Z1)
    
    if option_type == "call":
        payoff = np.maximum(S - K, 0)
    else:
        payoff = np.maximum(K - S, 0)
    
    return np.exp(-r * T) * np.mean(payoff)

print("✓ Fonctions Monte Carlo définies")

✓ Fonctions Monte Carlo définies


## 7. ÉTAPE 1: Téléchargement des Données Market

In [32]:
print("=" * 80)
print("ÉTAPE 1: Téléchargement des Données Market")
print("=" * 80)

# Récupérer le prix spot
S0 = fetch_spot(TICKER)
print(f"\nPrix spot {TICKER}: ${S0:.2f}")

# Télécharger toutes les options disponibles
calls_df, puts_df = download_all_options(TICKER, years_ahead=2.5)

print(f"\n✓ Téléchargement terminé")
print(f"  Calls: {len(calls_df)} options")
print(f"  Puts:  {len(puts_df)} options")
print(f"  Total: {len(calls_df) + len(puts_df)} options pour l'entraînement du NN")

ÉTAPE 1: Téléchargement des Données Market

Prix spot SPY: $662.52
Téléchargement de toutes les expirations jusqu'à 2.5 ans...
  Processed 5/31 expirations...
  Processed 10/31 expirations...
  Processed 15/31 expirations...
  Processed 20/31 expirations...
  Processed 25/31 expirations...
  Processed 30/31 expirations...

✓ Téléchargement terminé
  Calls: 1316 options
  Puts:  1259 options
  Total: 2575 options pour l'entraînement du NN


## 7b. Création des Grilles

In [33]:
# Créer les grilles pour les heatmaps
# MARKET: Utiliser les valeurs exactes de yfinance pour K et T
# HESTON: Utiliser les grilles synthétiques S0 ± 10

# Grille K pour MARKET: utiliser les strikes disponibles dans yfinance (S0 ± 10)
K_available_calls = sorted(calls_df['K'].unique())
K_available_puts = sorted(puts_df['K'].unique())
K_available = sorted(set(K_available_calls + K_available_puts))

# Filtrer pour garder uniquement K dans [S0-10, S0+10]
K_min, K_max = S0 - SPAN, S0 + SPAN
K_grid_market = np.array([k for k in K_available if K_min <= k <= K_max])

# Grille T pour MARKET: utiliser les maturités disponibles dans yfinance [0.1, 1.0]
T_available_calls = sorted(calls_df['T'].unique())
T_available_puts = sorted(puts_df['T'].unique())
T_available = sorted(set(T_available_calls + T_available_puts))

# Filtrer pour garder uniquement T dans [0.1, 1.0]
T_min, T_max = 0.1, 1.0
T_grid_market = np.array([t for t in T_available if T_min <= t <= T_max])

# Grilles synthétiques pour HESTON (S × K)
S_min, S_max = S0 - SPAN, S0 + SPAN
S_grid = np.arange(S_min, S_max + STEP, STEP)
K_grid_heston = np.arange(K_min, K_max + STEP, STEP)

print(f"\n✓ Grilles créées:")
print(f"\n  MARKET (valeurs yfinance):")
print(f"    K: {K_grid_market.min():.1f} à {K_grid_market.max():.1f}, {len(K_grid_market)} strikes disponibles")
print(f"    T: {T_grid_market.min():.2f} à {T_grid_market.max():.2f} ans, {len(T_grid_market)} maturités disponibles")
print(f"    Heatmap Market (K × T): {len(K_grid_market)} × {len(T_grid_market)} = {len(K_grid_market) * len(T_grid_market)} points")

print(f"\n  HESTON (grille synthétique):")
print(f"    S: {S_min:.1f} à {S_max:.1f}, {len(S_grid)} points (step {STEP})")
print(f"    K: {K_min:.1f} à {K_max:.1f}, {len(K_grid_heston)} points (step {STEP})")
print(f"    Heatmap Heston (S × K): {len(S_grid)} × {len(K_grid_heston)} = {len(S_grid) * len(K_grid_heston)} points")


✓ Grilles créées:

  MARKET (valeurs yfinance):
    K: 653.0 à 672.0, 20 strikes disponibles
    T: 0.10 à 0.86 ans, 14 maturités disponibles
    Heatmap Market (K × T): 20 × 14 = 280 points

  HESTON (grille synthétique):
    S: 652.5 à 672.5, 21 points (step 1.0)
    K: 652.5 à 672.5, 21 points (step 1.0)
    Heatmap Heston (S × K): 21 × 21 = 441 points


## 8. ÉTAPE 2: Heatmap des Prix Market

In [34]:
print("=" * 80)
print("ÉTAPE 2: Heatmap des Prix Market (K × T)")
print("=" * 80)

# Utiliser les grilles exactes de yfinance
call_prices_market = np.full((len(T_grid_market), len(K_grid_market)), np.nan)
put_prices_market = np.full((len(T_grid_market), len(K_grid_market)), np.nan)

print(f"\nMapping exact des prix market sur la grille K × T (valeurs yfinance)...")

for i, T in enumerate(T_grid_market):
    for j, K in enumerate(K_grid_market):
        # Trouver le prix exact pour (K, T)
        call_match = calls_df[(calls_df["K"] == K) & (calls_df["T"] == T)]
        put_match = puts_df[(puts_df["K"] == K) & (puts_df["T"] == T)]
        
        if len(call_match) > 0:
            call_prices_market[i, j] = call_match["C_mkt"].iloc[0]
        
        if len(put_match) > 0:
            put_prices_market[i, j] = put_match["P_mkt"].iloc[0]

num_call_prices = np.sum(~np.isnan(call_prices_market))
num_put_prices = np.sum(~np.isnan(put_prices_market))
total_points = len(T_grid_market) * len(K_grid_market)
print(f"\n✓ Prix market mappés")
print(f"  Calls: {num_call_prices}/{total_points} points ({100*num_call_prices/total_points:.1f}%)")
print(f"  Puts:  {num_put_prices}/{total_points} points ({100*num_put_prices/total_points:.1f}%)")

# Créer les heatmaps avec plotly
K_mesh_market, T_mesh_market = np.meshgrid(K_grid_market, T_grid_market)

fig_call_market = go.Figure(
    data=go.Heatmap(
        x=K_grid_market,
        y=T_grid_market,
        z=call_prices_market,
        colorscale="Viridis",
        colorbar=dict(title="Prix Call"),
    )
)
fig_call_market.update_layout(
    title=f"Prix Call Market - {TICKER} (K × T)",
    xaxis_title="Strike K",
    yaxis_title="Maturité T (années)",
    width=900, height=600
)
fig_call_market.show()

fig_put_market = go.Figure(
    data=go.Heatmap(
        x=K_grid_market,
        y=T_grid_market,
        z=put_prices_market,
        colorscale="Viridis",
        colorbar=dict(title="Prix Put"),
    )
)
fig_put_market.update_layout(
    title=f"Prix Put Market - {TICKER} (K × T)",
    xaxis_title="Strike K",
    yaxis_title="Maturité T (années)",
    width=900, height=600
)
fig_put_market.show()

print(f"\n✓ Heatmaps des prix market affichées")

ÉTAPE 2: Heatmap des Prix Market (K × T)

Mapping exact des prix market sur la grille K × T (valeurs yfinance)...

✓ Prix market mappés
  Calls: 163/280 points (58.2%)
  Puts:  159/280 points (56.8%)



✓ Heatmaps des prix market affichées


## 9. ÉTAPE 3: IV Surface des Données Market (3D)

In [35]:
print("=" * 80)
print("ÉTAPE 3: IV Surface Market (K × T) en 3D")
print("=" * 80)

# Utiliser les grilles exactes de yfinance
call_iv_market = np.full((len(T_grid_market), len(K_grid_market)), np.nan)
put_iv_market = np.full((len(T_grid_market), len(K_grid_market)), np.nan)

print(f"\nMapping exact des IV market sur la grille K × T (valeurs yfinance)...")

for i, T in enumerate(T_grid_market):
    for j, K in enumerate(K_grid_market):
        # Trouver l'IV exact pour (K, T)
        call_match = calls_df[(calls_df["K"] == K) & (calls_df["T"] == T)]
        put_match = puts_df[(puts_df["K"] == K) & (puts_df["T"] == T)]
        
        if len(call_match) > 0 and not pd.isna(call_match["iv_market"].iloc[0]):
            call_iv_market[i, j] = call_match["iv_market"].iloc[0]
        
        if len(put_match) > 0 and not pd.isna(put_match["iv_market"].iloc[0]):
            put_iv_market[i, j] = put_match["iv_market"].iloc[0]

num_call_iv = np.sum(~np.isnan(call_iv_market))
num_put_iv = np.sum(~np.isnan(put_iv_market))
total_points = len(T_grid_market) * len(K_grid_market)
print(f"\n✓ IV market mappées")
print(f"  Calls: {num_call_iv}/{total_points} points IV ({100*num_call_iv/total_points:.1f}%)")
print(f"  Puts:  {num_put_iv}/{total_points} points IV ({100*num_put_iv/total_points:.1f}%)")

# Surface 3D avec meshgrid
K_mesh_market, T_mesh_market = np.meshgrid(K_grid_market, T_grid_market)

fig_call_iv_market_3d = go.Figure(
    data=go.Surface(
        x=K_mesh_market,
        y=T_mesh_market,
        z=call_iv_market,
        colorscale="Viridis",
        colorbar=dict(title="IV Call"),
    )
)
fig_call_iv_market_3d.update_layout(
    title=f"IV Surface Market Call - {TICKER} (K × T)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Maturité T (années)",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_call_iv_market_3d.show()

fig_put_iv_market_3d = go.Figure(
    data=go.Surface(
        x=K_mesh_market,
        y=T_mesh_market,
        z=put_iv_market,
        colorscale="Viridis",
        colorbar=dict(title="IV Put"),
    )
)
fig_put_iv_market_3d.update_layout(
    title=f"IV Surface Market Put - {TICKER} (K × T)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Maturité T (années)",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_put_iv_market_3d.show()

print(f"\n✓ Surfaces IV market 3D affichées")

ÉTAPE 3: IV Surface Market (K × T) en 3D

Mapping exact des IV market sur la grille K × T (valeurs yfinance)...



✓ IV market mappées
  Calls: 163/280 points IV (58.2%)
  Puts:  159/280 points IV (56.8%)



✓ Surfaces IV market 3D affichées


## 10. ÉTAPE 4: Calibration Heston (Neural Network)

In [ ]:
print("=" * 80)
print("ÉTAPE 4: Calibration Heston via PyTorch")
print("=" * 80)

calib = calibrate_heston(calls_df, RF_RATE, 0.0, MAX_POINTS, MAX_ITERS, LR)

print("\n" + "=" * 80)
print("PARAMÈTRES HESTON CALIBRÉS")
print("=" * 80)
for k, v in calib.items():
    print(f"  {k:6s} = {v:.6f}")
print("=" * 80)

params_tensor = params_from_calib(calib)

ÉTAPE 4: Calibration Heston via PyTorch
Calibration sur 841 points...
Lancement de l'entraînement du réseau de neurones...



  🔄 NN Training | Epoch  10/150 | Loss = 9.542552e+03


## 11. ÉTAPE 5: Heatmap des Prix Heston (Monte Carlo, S × K)

In [ ]:
print("=" * 80)
print("ÉTAPE 5: Calcul des prix Heston via Monte Carlo (S × K)")
print("=" * 80)

call_prices_heston = np.zeros((len(S_grid), len(K_grid_heston)))
put_prices_heston = np.zeros((len(S_grid), len(K_grid_heston)))

# Fixer T à 1 an pour les heatmaps Heston
T_heston = 1.0

print(f"\nCalcul des prix Heston par Monte Carlo (T = {T_heston} an)...")
print(f"Grille: {len(S_grid)} spots × {len(K_grid_heston)} strikes = {len(S_grid) * len(K_grid_heston)} prix à calculer")

for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid_heston):
        call_prices_heston[i, j] = heston_mc_pricer(
            S, K, T_heston, r, v0, theta, kappa, sigma_v, rho, 
            n_paths=50000, n_steps=100, option_type="call"
        )
        put_prices_heston[i, j] = heston_mc_pricer(
            S, K, T_heston, r, v0, theta, kappa, sigma_v, rho, 
            n_paths=50000, n_steps=100, option_type="put"
        )
    
    if (i + 1) % 5 == 0 or i == len(S_grid) - 1:
        print(f"  Progress: {i+1}/{len(S_grid)} spots calculés ({100*(i+1)/len(S_grid):.0f}%)")

print(f"\n✓ Prix Heston calculés par Monte Carlo")

# Créer les heatmaps Heston
S_mesh, K_mesh_heston = np.meshgrid(S_grid, K_grid_heston, indexing="ij")

fig_call_heston = go.Figure(
    data=go.Heatmap(
        x=K_grid_heston,
        y=S_grid,
        z=call_prices_heston,
        colorscale="Viridis",
        colorbar=dict(title="Prix Call Heston"),
    )
)
fig_call_heston.update_layout(
    title=f"Prix Call Heston (MC) - T={T_heston} an (S × K)",
    xaxis_title="Strike K",
    yaxis_title="Spot S",
    width=900, height=600
)
fig_call_heston.show()

fig_put_heston = go.Figure(
    data=go.Heatmap(
        x=K_grid_heston,
        y=S_grid,
        z=put_prices_heston,
        colorscale="Viridis",
        colorbar=dict(title="Prix Put Heston"),
    )
)
fig_put_heston.update_layout(
    title=f"Prix Put Heston (MC) - T={T_heston} an (S × K)",
    xaxis_title="Strike K",
    yaxis_title="Spot S",
    width=900, height=600
)
fig_put_heston.show()

print(f"\n✓ Heatmaps des prix Heston affichées")

## 12. ÉTAPE 6: IV Surface BS Inversée (S × K) en 3D

In [ ]:
print("=" * 80)
print("ÉTAPE 6: Calcul des IV BS à partir des prix Heston (S × K)")
print("=" * 80)

call_iv_heston = np.zeros((len(S_grid), len(K_grid_heston)))
put_iv_heston = np.zeros((len(S_grid), len(K_grid_heston)))

print(f"\nInversion des prix Heston en IV BS...")

for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid_heston):
        call_price = call_prices_heston[i, j]
        put_price = put_prices_heston[i, j]
        
        try:
            call_iv_heston[i, j] = implied_volatility(call_price, S, K, T_heston, r, "call")
        except:
            call_iv_heston[i, j] = np.nan
        
        try:
            put_iv_heston[i, j] = implied_volatility(put_price, S, K, T_heston, r, "put")
        except:
            put_iv_heston[i, j] = np.nan
    
    if (i + 1) % 5 == 0 or i == len(S_grid) - 1:
        print(f"  Progress: {i+1}/{len(S_grid)} spots calculés ({100*(i+1)/len(S_grid):.0f}%)")

num_call_iv_heston = np.sum(~np.isnan(call_iv_heston))
num_put_iv_heston = np.sum(~np.isnan(put_iv_heston))
total_heston = len(S_grid) * len(K_grid_heston)
print(f"\n✓ IV BS calculées")
print(f"  Calls: {num_call_iv_heston}/{total_heston} IV ({100*num_call_iv_heston/total_heston:.1f}%)")
print(f"  Puts:  {num_put_iv_heston}/{total_heston} IV ({100*num_put_iv_heston/total_heston:.1f}%)")

# Surface 3D
S_mesh, K_mesh_heston = np.meshgrid(S_grid, K_grid_heston, indexing="ij")

fig_call_iv_heston_3d = go.Figure(
    data=go.Surface(
        x=K_grid_heston,
        y=S_grid,
        z=call_iv_heston,
        colorscale="Viridis",
        colorbar=dict(title="IV Call BS"),
    )
)
fig_call_iv_heston_3d.update_layout(
    title=f"IV Surface Call (BS inversé depuis Heston) - T={T_heston} an (S × K)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Spot S",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_call_iv_heston_3d.show()

fig_put_iv_heston_3d = go.Figure(
    data=go.Surface(
        x=K_grid_heston,
        y=S_grid,
        z=put_iv_heston,
        colorscale="Viridis",
        colorbar=dict(title="IV Put BS"),
    )
)
fig_put_iv_heston_3d.update_layout(
    title=f"IV Surface Put (BS inversé depuis Heston) - T={T_heston} an (S × K)",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Spot S",
        zaxis_title="IV"
    ),
    width=900, height=700
)
fig_put_iv_heston_3d.show()

print(f"\n✓ Surfaces IV Heston (BS inversé) affichées en 3D")

## 13. Résumé et Statistiques

In [ ]:
print("=" * 80)
print("RÉSUMÉ FINAL")
print("=" * 80)

print(f"\nPrix Heston Call:")
print(f"  Min:  {np.nanmin(call_prices_heston):.4f}")
print(f"  Max:  {np.nanmax(call_prices_heston):.4f}")
print(f"  Mean: {np.nanmean(call_prices_heston):.4f}")

print(f"\nPrix Heston Put:")
print(f"  Min:  {np.nanmin(put_prices_heston):.4f}")
print(f"  Max:  {np.nanmax(put_prices_heston):.4f}")
print(f"  Mean: {np.nanmean(put_prices_heston):.4f}")

print(f"\nIV Call (prix Heston → BS):")
print(f"  Min:  {np.nanmin(call_iv_heston):.4f}")
print(f"  Max:  {np.nanmax(call_iv_heston):.4f}")
print(f"  Mean: {np.nanmean(call_iv_heston):.4f}")

print(f"\nIV Put (prix Heston → BS):")
print(f"  Min:  {np.nanmin(put_iv_heston):.4f}")
print(f"  Max:  {np.nanmax(put_iv_heston):.4f}")
print(f"  Mean: {np.nanmean(put_iv_heston):.4f}")

print("\n" + "=" * 80)
print("✓ ANALYSE COMPLÈTE TERMINÉE")
print("=" * 80)

## 14. Export des Résultats

In [ ]:
# Créer DataFrame avec tous les résultats
results = []
for i, S in enumerate(S_grid):
    for j, K in enumerate(K_grid):
        results.append({
            'S': S,
            'K': K,
            'call_price_heston': call_prices_heston[i, j],
            'put_price_heston': put_prices_heston[i, j],
            'call_iv_heston': call_iv_heston[i, j],
            'put_iv_heston': put_iv_heston[i, j],
        })

df_results = pd.DataFrame(results)
filename = f'heston_complete_analysis_{TICKER}.csv'
df_results.to_csv(filename, index=False)

print(f"✓ Résultats exportés vers: {filename}")
display(df_results.head(10))